In [ ]:
# 1.ADIM : Google Drive'ı bağla
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

print("StyleGAN'ın [75-78] yardımcı kütüphaneleri (ninja, psutil) kuruluyor.")
!pip install -q ninja psutil

print("\nADIM 1 TAMAMLANDI. (Oturumu yeniden başlatmamıza gerek YOK)")

In [ ]:
# 2. ADIM: Drive Bağlantısı ve DÜZELTİLMİŞ Kodun Klonlanması
from google.colab import drive
import os

print("Drive'ı bağlıyorum ve proje klasörüme geçiyorum...")
drive.mount('/content/drive')

proje_yolu = "/content/drive/MyDrive/TUBITAK_Projesi/"
os.chdir(proje_yolu)
!pwd # Kontrol

# GPU Kontrolü
print("\nGPU Kontrolü (A100/V100 [124-126] olmalı):")
!nvidia-smi -L


# Hata veren 'NVlabs' [1102-1111, 1113-1123, 1125-1134, 1136-1145] deposunu değil,
# 'derivative' [user] hatasını [user] çözen güncellenmiş forku klonluyorum.
stylegan_klasoru_adi = "stylegan2-ada-pytorch-fixed" # (Yeni klasör adı)

if not os.path.exists(stylegan_klasoru_adi):
    print(f"\nGüncellenmiş (Hata Düzeltilmiş) StyleGAN2-ADA [75-78] deposu klonlanıyor...")
    # Bu depo, PyTorch 2.x [user] ile çalışacak şekilde düzeltilmiştir
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch.git {stylegan_klasoru_adi}
else:
    print(f"\nGüncellenmiş StyleGAN2-ADA [75-78] deposu ('{stylegan_klasoru_adi}') zaten mevcut.")

# Yeni klonlanan klasörün içine girelim
stylegan_klasoru = os.path.join(proje_yolu, stylegan_klasoru_adi)
os.chdir(stylegan_klasoru)

print("\nADIM 2 TAMAMLANDI. ADIM 3'e geçebilirsiniz.")

In [ ]:
# 3. ADIM: Veri Setini Hazırlama ve StyleGAN Formatına Dönüştürme
print("ADIM 3 (Veri Hazırlama + Formatlama) BAŞLATILDI")

import zipfile
import shutil
from PIL import Image

# Yolları tanımla
proje_yolu = "/content/drive/MyDrive/TUBITAK_Projesi/"
veri_zip_yolu = os.path.join(proje_yolu, "dataset_son_hali_256_v1.zip")
veri_klasoru_adi = "dataset_egitim_256"
veri_klasoru_yolu = os.path.join(proje_yolu, veri_klasoru_adi)

# Klasörleri temizle ve oluştur
print(f"'{veri_klasoru_adi}' klasörü hazırlanıyor...")
if os.path.exists(veri_klasoru_yolu): shutil.rmtree(veri_klasoru_yolu)
os.makedirs(veri_klasoru_yolu, exist_ok=True)

# ZIP dosyasını aç
print(f"'{veri_zip_yolu}' (613 resim) [image_2b189c.png, image_2b88fe.png] Colab ortamına açılıyor...")
if os.path.exists(veri_zip_yolu):
    with zipfile.ZipFile(veri_zip_yolu, 'r') as zip_ref:
        zip_ref.extractall(veri_klasoru_yolu)
    print("Veri seti başarıyla açıldı.")
else:
    print(f"KRİTİK HATA: '{veri_zip_yolu}' bulunamadı. Lütfen Google Drive'ınızı kontrol edin.")
    exit()

print("Veri seti StyleGAN formatına dönüştürülüyor (.zip)...")
# (Doğru klasördeyiz: 'stylegan2-ada-pytorch-fixed')
!python dataset_tool.py --source={veri_klasoru_yolu} --dest={proje_yolu}/empresyonizm_256.zip

print("\nADIM 3 TAMAMLANDI. Veri seti hazırlandı.")

In [ ]:
# 4. ADIM: Eğitimi Başlatma
print(" 4.Adım : MODEL EĞİTİMİ BAŞLATILIYOR!")


# Gerekli değişkenleri tanımla
proje_yolu = "/content/drive/MyDrive/TUBITAK_Projesi/"
OUTDIR = os.path.join(proje_yolu, "results")
DATA = os.path.join(proje_yolu, "empresyonizm_256.zip")
RESUME_PKL = "https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/transfer-learning-source-nets/ffhq-res256-mirror-paper256-noaug.pkl"
GPUS = 1
KIMG = 2000
GAMMA = 1
BATCH_SIZE = 16 # (GPU Hafıza hatasını önlemek için)

print(f"Veri Seti: {DATA}")
print(f"Sonuçlar buraya kaydedilecek: {OUTDIR}")
print("Artık GÜNCELLENMİŞ StyleGAN kodu ve STABİL PyTorch versiyonundayız. Eğitim başlıyor.")

# Eğitimi başlat
!python train.py --outdir={OUTDIR} --data={DATA} --resume={RESUME_PKL} --gpus={GPUS} --kimg={KIMG} --gamma={GAMMA} --batch={BATCH_SIZE}